In [11]:
import setup

setup.init_django()

In [12]:
from market.models import StockQuote

In [13]:
from django.db.models import Avg, F, RowRange, Window, Max
from django.db.models.functions import TruncDate
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [14]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now

qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

2932

In [15]:
total = 0
for obj in qs:
    total += obj.close_price

In [16]:
total / qs.count()

Decimal('570.2463719645293315143246930')

In [17]:
qs.values("company").annotate(avg_price=Avg("close_price"))

<QuerySet [{'company': 2, 'avg_price': Decimal('570.2463719645293315')}]>

In [18]:
count = 5
ticker = "AAPL"
rolling_qs = list(qs.filter(company__ticker=ticker).order_by("-time")[:count])
rolling_qs.reverse()
rolling_qs

[]

In [19]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = total / (i + 1)
    print(i + 1, obj.id, obj.close_price, avg)

In [20]:
latest_daily_timestamps = (
    StockQuote.objects.filter(
        company__ticker=ticker, time__range=(start_date - timedelta(days=40), end_date)
    )
    .annotate(date=TruncDate("time"))
    .values("company", "date")
    .annotate(latest_time=Max("time"))
    .values("company", "date", "latest_time")
    .order_by("date")
)

actual_timestamps = [x["latest_time"] for x in latest_daily_timestamps]

In [21]:
qs = StockQuote.objects.filter(
    company__ticker=ticker,
    time__range=(start_date, end_date),
    time__in=actual_timestamps,
)

In [22]:
frame_start = -(count - 1)
ma_val = qs.annotate(
    ma=Window(
        expression=Avg("close_price"),
        order_by=F("time").asc(),
        partition_by=[],
        frame=RowRange(start=frame_start, end=0),
    )
).order_by("-time")

In [23]:
for obj in ma_val[:5]:
    print(obj.id, obj.close_price, obj.ma, obj.time)

In [ ]:
frame_start = -(count - 1)
ma_vals = (
    qs.annotate(
        ma_5=Window(
            expression=Avg("close_price"),
            order_by=F("time").asc(),
            partition_by=[],
            frame=RowRange(start=-4, end=0),
        ),
        ma_20=Window(
            expression=Avg("close_price"),
            order_by=F("time").asc(),
            partition_by=[],
            frame=RowRange(start=-19, end=0),
        ),
    )
    .order_by("-time")
    .first()
)

ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20